In [1]:
import numpy as np
from scipy.sparse.linalg import svds
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linprog
from itertools import combinations
from sklearn.cluster import KMeans

In [2]:
def score(K, K0, m, D, scatterplot=False):
    """
    Translates the given R function to Python.

    Parameters:
        K: Number of singular values/vectors to compute.
        K0: Number of vertices to estimate.
        m: Parameter for vertices_est function.
        D: Input matrix.
        scatterplot: Whether to generate scatterplot.
    
    Returns:
        A dictionary containing A_hat, R, V, Pi, and theta.
    """
    # Dimensions
    p = D.shape[0]
    
    # Step 1: Compute SVD
    u, _, _ = svds(D, k=K)
    Xi = u
    print('Got here 0')

    # Make the first column of Xi non-negative
    Xi[:, 0] = np.abs(Xi[:, 0])
    R = np.apply_along_axis(lambda x: x / Xi[:, 0], axis=0, arr=Xi[:, 1:K])
    print('Got here 1')
    # Step 2: Estimate vertices
    vertices_est_obj = vertices_est(R, K0, m)
    V = vertices_est_obj["V"]
    theta = vertices_est_obj["theta"]
    print('Got here 2')

    if scatterplot:
        import matplotlib.pyplot as plt
        plt.scatter(R[:, 0], R[:, 1])
        plt.scatter(V[:, 0], V[:, 1], color='red', s=50)
        plt.show()

    # Step 3: Compute Pi
    Pi = np.hstack([R, np.ones((p, 1))]) @ np.linalg.pinv(np.hstack([V, np.ones((K, 1))]))
    Pi = np.maximum(Pi, 0)  # Element-wise max
    temp = np.sum(Pi, axis=1, keepdims=True)
    Pi = Pi / temp

    # Step 4: Compute A_hat
    A_hat = Xi[:, [0]] * Pi

    # Step 5: Normalize rows of A_hat
    temp = np.sum(A_hat, axis=0, keepdims=True)
    A_hat = A_hat / temp

    return {"A_hat": A_hat, "R": R, "V": V, "Pi": Pi, "theta": theta}


In [3]:

def vertices_est(R, K0, m):
    """
    Estimate vertices based on the R matrix.

    Parameters:
        R: Input matrix (n x (K-1)).
        K0: Number of vertices to estimate.
        m: Number of clusters for k-means.
    
    Returns:
        A dictionary with `V` (estimated vertices) and `theta` (cluster centers).
    """
    print('Got here')
    K = R.shape[1] + 1

    # Step 2a: k-means clustering
    kmeans = KMeans(n_clusters=m, max_iter=K * 100, n_init=K * 10)
    kmeans.fit(R)
    theta = kmeans.cluster_centers_
    theta_original = np.copy(theta)
    
    # Optional: Visualization (if needed)
    # import matplotlib.pyplot as plt
    # plt.scatter(R[:, 0], R[:, 1], alpha=0.5)
    # plt.scatter(theta[:, 0], theta[:, 1], color='red', s=50)
    # plt.show()

    # Step 2b': Find the two furthest apart centers
    distance_matrix = cdist(theta, theta, 'sqeuclidean')
    top2_indices = np.unravel_index(np.argmax(distance_matrix, axis=None), distance_matrix.shape)
    theta0 = theta[list(top2_indices), :]
    theta = np.delete(theta, list(top2_indices), axis=0)
    
    # Step 2b: Select K0 vertices
    if K0 > 2:
        for _ in range(3, K0 + 1):
            distance_to_theta0 = cdist(theta, theta0).mean(axis=1)
            max_index = np.argmax(distance_to_theta0)
            theta0 = np.vstack([theta0, theta[max_index]])
            theta = np.delete(theta, max_index, axis=0)
        theta = theta0

    # Step 2c: Find optimal combination of K vertices
    combs = np.array(list(combinations(range(K0), K)))
    max_values = np.zeros(len(combs))

    for i, comb in enumerate(combs):
        for j in range(K0):
            max_values[i] = max(simplex_dist(theta[j], theta[comb]), max_values[i])
    
    min_index = np.argmin(max_values)

    return {"V": theta[combs[min_index[0]]], "theta": theta_original}


def simplex_dist(theta, V):
    """
    Calculate simplex distance between theta and vertices V.

    Parameters:
        theta: A single point.
        V: Vertices of the simplex.

    Returns:
        Distance value.
    """
    K_minus_1 = V.shape[0] - 1

    # Construct VV and D matrices
    VV = np.vstack([np.eye(K_minus_1), -np.ones((1, K_minus_1))]) @ V
    D = VV @ VV.T
    d = VV @ (theta - V[-1])

    # Construct inequality constraints
    A = np.vstack([np.eye(K_minus_1), -np.ones((1, K_minus_1))])
    b = np.zeros(K_minus_1 + 1)
    b[-1] = -1

    # Solve quadratic programming problem using linear approximation
    result = linprog(c=d, A_ub=A, b_ub=b, bounds=(None, None), method='highs')

    if not result.success:
        raise ValueError("Optimization failed!")

    return np.sum((theta - V[-1]) ** 2) + 2 * result.fun


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd

# Download NLTK resources if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mafuangimemkamon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mafuangimemkamon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mafuangimemkamon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df = pd.read_csv('processed_data/stat_courses.csv')
df = df[df['filename'].str.contains('STAT')|df['text'].str.lower().str.contains('statistics')]
df


,filename,text,cleaned_text
0,IGA 451M - 2020 Spring 1 (170371).pdf,"IGA-451M: CONTROVERSIES IN CLIMATE, ENERGY \n&...","IGA-451M: CONTROVERSIES IN CLIMATE, ENERGY & T..."
1,STAT 123 - 2022 Spring (126048).pdf,"Thank\tyou\tfor\ttaking\tStat\t123\t2022,\tQua...","Thank you for taking Stat 123 2022, Quantitati..."
2,STAT 364 - 2022 Spring (214539).pdf,Stat364 : Scalable Statistical Inference for B...,Stat364 : Scalable Statistical Inference for B...
3,PSY 2030 - 2021 Fall (160667).pdf,Bayesian Data Analysis\nPsych 2030 { Fall 2021...,Bayesian Data Analysis Psych 2030 { Fall 2021 ...
4,EPI 224 - 2024 Fall 2 (190291).pdf,\nEPI224 Cancer Prevention (2023 Fall 2) Syl...,EPI224 Cancer Prevention (2023 Fall 2) Syllab...
...,...,...,...
242,STAT 171 - 2024 Spring (113721).pdf,"Statistics 171: Stochastic Processes, Spring 2...","Statistics 171: Stochastic Processes, Spring 2..."
245,STAT 139 - 2024 Fall (110751).pdf,STAT 139: Introduction to Linear Models\nFall ...,STAT 139: Introduction to Linear Models Fall 2...
246,STAT 221 - 2020 Fall (115077).pdf,Fall 2020\nSTAT 221: Computational Tools for S...,Fall 2020 STAT 221: Computational Tools for St...
247,BIOSTAT 241_BST 241 - 2024 Spring (190066).pdf,BST 241/ BIOSTAT 241: Inference II\n Spring 2...,BST 241/ BIOSTAT 241: Inference II Spring 202...


In [6]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming `df['cleaned_text']` contains the documents
documents = df['cleaned_text'].values
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]  # Lemmatize and remove non-alphabetic tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
    return ' '.join(tokens)  # Join tokens back into a single string

# Preprocess each document
processed_docs = [preprocess(doc) for doc in documents]

# Convert documents to a TF-IDF matrix
vectorizer = TfidfVectorizer(max_features=100)  # Keep only top 1000 features
tfidf_matrix = vectorizer.fit_transform(processed_docs)

# Feature names and matrix shape
feature_names = vectorizer.get_feature_names_out()
matrix_shape = tfidf_matrix.shape

print(f"TF-IDF matrix shape: {matrix_shape}")
print(f"First 5 feature names: {feature_names[:5]}")

# Convert to DataFrame if needed
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
print(tfidf_df.head())


TF-IDF matrix shape: (148, 100)
First 5 feature names: ['academic' 'al' 'also' 'analysis' 'assignment']
   academic        al      also  analysis  assignment  available    canvas  \
0  0.004526  0.023590  0.011909  0.008162    0.026299   0.001877  0.004128   
1  0.000000  0.000000  0.000000  0.000000    0.000000   0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.077060    0.000000   0.000000  0.000000   
3  0.052626  0.000000  0.039560  0.037959    0.000000   0.000000  0.047994   
4  0.071688  0.105077  0.033681  0.025854    0.110638   0.029730  0.089896   

       care      case  causal  ...      time      tool     topic       two  \
0  0.000000  0.002228     0.0  ...  0.083735  0.000000  0.009213  0.007298   
1  0.000000  0.000000     0.0  ...  0.086658  0.000000  0.000000  0.000000   
2  0.000000  0.000000     0.0  ...  0.076013  0.000000  0.000000  0.000000   
3  0.000000  0.000000     0.0  ...  0.037444  0.000000  0.107108  0.000000   
4  0.026974  0.017642     0.0  ...  0

In [ ]:
score(6, 3, 3, tfidf_df.values, scatterplot=True)